In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Semester/MTP/Phishing-Detection/

/content/drive/MyDrive/Semester/MTP/Phishing-Detection


In [3]:
# !pip install tldextract
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199772 sha256=9ac3acfb09aaf115f8349adca6b4739697a0b7af04d72fb6f4075d720e931865
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [8]:
from urllib.parse import urlparse
# import tldextract

import numpy as np
import pandas as pd
from gensim.models import Word2Vec
import fasttext
# import fasttext.util
# import spacy

In [ ]:
phish_df = pd.read_csv('/content/drive/MyDrive/Semester/MTP/Phishing-Detection/dataset/phishtank_data_corpus_complete_15-07-23.csv')
nonphish_df = pd.read_csv('/content/drive/MyDrive/Semester/MTP/Phishing-Detection/dataset/original_website_dataset.csv')

In [ ]:
phish_df.head()

,id,url,is_valid,phishing_url,submission_time
0,8225238,https://phishtank.org/phish_detail.php?phish_i...,Unknown,https://pocztapolsk.buzz/pay,Jul 15th 2023 9:45 AM
1,8225237,https://phishtank.org/phish_detail.php?phish_i...,VALID PHISH,https://alertsuser.github.io/pantek/kimax/,Jul 15th 2023 9:38 AM
2,8225236,https://phishtank.org/phish_detail.php?phish_i...,Unknown,https://digicamforums.com,Jul 15th 2023 9:19 AM
3,8225235,https://phishtank.org/phish_detail.php?phish_i...,Unknown,SPAMs://wirelessworldshop.com/,Jul 15th 2023 9:19 AM
4,8225234,https://phishtank.org/phish_detail.php?phish_i...,Unknown,SPAMs://www.wirelessworldshop.com,Jul 15th 2023 9:19 AM


In [ ]:
nonphish_df.head()

,url,Speical_Char,Have_IP,Have_At,URL_length,URL_Depth,redirection,time_get_redirect,port_in_url,use_http,...,unescape,escape,ActiveXObject,fromCharCode,atob,Punny_Code,TLDs,Title,country_name,label
0,https://sites.google.com/site/policyclaming76745/,3,0,0,0,2,6,0,0,0,...,0,0,0,0,0,0,com,Google Sites,US,1
1,https://www.pinterest.com/abbiestever/?redirec...,5,0,0,0,1,6,0,0,0,...,0,0,0,0,0,0,com,Abbie Roose Stever (abbiestever) - Profile | P...,NaN,0
2,https://www.pinterest.com/abiolatv/bombshell-g...,3,0,0,1,2,6,0,0,0,...,0,0,0,0,0,0,com,No Title,NaN,0
3,https://balajipackersguntur.com/images/,2,0,0,0,1,6,0,0,0,...,0,0,0,0,0,0,com,Index of /images,IN,1
4,https://sites.google.com/site/claming564336670...,3,0,0,0,2,6,0,0,0,...,0,0,0,0,0,0,com,SECURITY,NaN,1


In [ ]:
phish_df.shape, nonphish_df.shape

((99923, 5), (167872, 32))

In [ ]:
df_1 = pd.DataFrame({'url': phish_df['phishing_url'], 'label': 1})
df_2 = pd.DataFrame({'url': nonphish_df['url'], 'label': 2})

In [ ]:
df = pd.concat([df_1, df_2]).sample(frac=1, ignore_index=True)

In [ ]:
df.head()

,url,label
0,https://82372932-cnfrmtion-etnol.start.page/,1
1,https://byochange.org/wp-admin/css/colors/ligh...,2
2,http://foreverhomesbyjen.com/index.php,1
3,https://vh420.timeweb.ru/blocked/?ref=co42204....,1
4,https://demodrop.com,2


In [ ]:
# 1 = phishing
# 2 = non-phishing
df['label'].value_counts()

2    167872
1     99923
Name: label, dtype: int64

In [ ]:
df.shape

(267795, 2)

In [ ]:
def filter_df(x):
    x = str(x).lower()
    try:
        scheme = urlparse(str(x))._asdict()['netloc']
        if scheme=='':
            return np.nan
        return x
    except:
        scheme = urlparse(str(x)[:-1])._asdict()['netloc']
        if scheme=='':
            return np.nan
        return x

In [ ]:
df['url'] = df['url'].apply(lambda x: filter_df(x))
df.dropna(inplace=True)

In [ ]:
df.shape

(267639, 2)

In [ ]:
def parser(x, key, type):
    try:
        scheme = urlparse(str(x))._asdict()[key]
    except:
        scheme = urlparse(str(x)[:-1])._asdict()[key]

    return pd.Series([scheme], index=[type])


def parse_domain(x):
    subdomain = None
    domain = None
    tld = None
    try:
        scheme = urlparse(str(x))._asdict()['netloc']
        url = tldextract.extract(scheme)
        subdomain, domain, tld = url.subdomain, url.domain, url.suffix
    except:
        scheme = urlparse(str(x)[:-1])._asdict()['netloc']
        url = tldextract.extract(scheme)
        subdomain, domain, tld = url.subdomain, url.domain, url.suffix

    # tld = tld.replace('.', ' ')
    return pd.Series([subdomain, domain, tld], index=['subdomain', 'domain', 'tld'])

In [ ]:
df_protocol = df['url'].apply(lambda x: parser(x, 'scheme', 'protocol'))
df_domain = df['url'].apply(lambda x: parse_domain(x))
df_path = df['url'].apply(lambda x: parser(x, 'path', 'path'))
df_url_length = df['url'].apply(lambda x: len(x)).rename('url_length')

In [ ]:
filtered_df = pd.concat([df_protocol, df_domain, df_path, df_url_length, df], axis=1)

In [ ]:
filtered_df.head()

,protocol,subdomain,domain,tld,path,url_length,url,label
0,https,,volkswagen,es,,21,https://volkswagen.es,2
1,https,,tipsofwork,com,,22,https://tipsofwork.com,2
2,https,,nofrag,com,,18,https://nofrag.com,2
3,https,f5ew3x,firebaseapp,com,/,31,https://f5ew3x.firebaseapp.com/,1
4,https,,lapresse,it,,19,https://lapresse.it,2


In [ ]:
processed_urls = filtered_df[['protocol', 'subdomain', 'domain', 'tld']].apply(' '.join, axis=1)

In [ ]:
processed_urls.head()

0            https  volkswagen es
1           https  tipsofwork com
2               https  nofrag com
3    https f5ew3x firebaseapp com
4              https  lapresse it
dtype: object

In [ ]:
processed_url_df = pd.DataFrame({'label': filtered_df['label'], 'processed_url': processed_urls})

In [ ]:
processed_url_df

,label,processed_url
0,2,https volkswagen es
1,2,https tipsofwork com
2,2,https nofrag com
3,1,https f5ew3x firebaseapp com
4,2,https lapresse it
...,...,...
267790,1,https www.innovalenligne legrand fr
267791,1,http acingyourfinances com
267792,2,https itspurts com
267793,2,https stromecov com


In [ ]:
# !ls /content/drive/MyDrive/Semester/MTP/Phishing-Detection/

cc.en.300.bin			  Embedding.ipynb
combined_embedding.npy		  PhishDetect_ann_basemodel.ipynb
dataset				  PhishDetect.ipynb
embedding_data_corpus_230823.txt  PhishDetect_lstm.ipynb
embedding_df.pkl


In [ ]:
processed_url_df = pd.read_csv('/content/drive/MyDrive/Semester/MTP/Phishing-Detection/embedding_data_corpus_230823.txt', names=['label', 'processed_url'])

In [ ]:
# preprocessing url -> converting to lowercase, replacing double space to single space
processed_url_df['processed_url'] = processed_url_df['processed_url'].apply(lambda x: x.lower().replace('  ',' '))

In [ ]:
processed_url_df.head()

,label,processed_url
0,2,https slshotels com
1,2,https svea com
2,2,https a901e00d-325c-4bcb-8767-8c733ed27a82.id ...
3,2,http postsfb-9gi0ywscbc novitium ca
4,1,http genuine-biscochitos-0174c6 netlify app


In [ ]:
# !rm /content/drive/MyDrive/Semester/MTP/Phishing-Detection/embedding_data_corpus_230823.txt

# FastText

### Supervised

In [ ]:
processed_url_df.to_csv('/content/drive/MyDrive/Semester/MTP/Phishing-Detection/embedding_data_corpus_230823.txt', columns=['label', 'processed_url'], header=None, index=False)

In [ ]:
%ll -h /content/drive/MyDrive/Semester/MTP/Phishing-Detection/

total 8.8G
-rw------- 1 root 6.8G Aug 23 10:58 cc.en.300.bin
-rw------- 1 root 1.4G Aug 15 01:25 combined_embedding.npy
drwx------ 3 root 4.0K Aug  5 19:28 dataset/
-rw------- 1 root 7.1M Aug 24 13:38 embedding_data_corpus_230823.txt
-rw------- 1 root 706M Aug 15 01:32 embedding_df.pkl
-rw------- 1 root  70K Aug 24 13:38 Embedding.ipynb
-rw------- 1 root 146K Aug 16 09:55 PhishDetect_ann_basemodel.ipynb
-rw------- 1 root 140K Aug  7 20:35 PhishDetect.ipynb
-rw------- 1 root 126K Aug 17 16:47 PhishDetect_lstm.ipynb


In [ ]:
model = fasttext.train_supervised('/content/drive/MyDrive/Semester/MTP/Phishing-Detection/embedding_data_corpus_230823.txt', dim=150, wordNgrams=3, lr=0.01, epoch=50)

In [ ]:
# model.get_word_vector('https')

In [ ]:
model.get_nearest_neighbors('http')

[(0.3461095988750458, 'polkadotchair'),
 (0.33669349551200867, 'fb-business-case-appeal-1284'),
 (0.33045050501823425,
  'bafybeic7fwmtxh2ti5b73kt6fk4ghdk3wz44nwmey63phed2rqhml7npci.ipfs'),
 (0.3294694423675537, 'befrienders'),
 (0.32063430547714233, 'bluelabel-hk'),
 (0.3202481269836426, 'poorcreditpaydayloans'),
 (0.3157230615615845, 'grupo-robo-do-pix'),
 (0.3135088086128235, 'morrisville'),
 (0.3132322430610657, 'parvo'),
 (0.31295454502105713, 'prometric')]

In [ ]:
len(model.words)

182233

In [ ]:
model.save_model('/content/drive/MyDrive/Semester/MTP/Phishing-Detection/phishing.custom.embedding.150.bin')

In [ ]:
ft = fasttext.load_model('/content/drive/MyDrive/Semester/MTP/Phishing-Detection/cc.en.300.bin')

In [ ]:
ft.get_dimension()

300

In [ ]:
fasttext.util.reduce_model(ft, 100)

In [ ]:
len(ft.words)

2000000

In [ ]:
ft.get_dimension()

300

In [ ]:
ft.get_nearest_neighbors('https')

[(0.8263505101203918, 'http'),
 (0.7738019824028015, '-https'),
 (0.7666537761688232, '.https'),
 (0.7135017514228821, 'here.https'),
 (0.7131780982017517, 'hhttp'),
 (0.702477753162384, 'too.https'),
 (0.7001454830169678, 'thttp'),
 (0.6946541666984558, 'below.https'),
 (0.6939340233802795, 'herehttps'),
 (0.6935752034187317, 'it.https')]

### unsupervised

In [ ]:
# processed_url_df.head()

In [ ]:
# processed_url_df.to_csv('/content/drive/MyDrive/Semester/MTP/Phishing-Detection/unsupervised_embedding_data_corpus_080923.txt', columns=['processed_url'], header=None, index=False)

In [9]:
model = fasttext.train_unsupervised('/content/drive/MyDrive/Semester/MTP/Phishing-Detection/unsupervised_embedding_data_corpus_080923.txt', dim=200, lr=0.01, epoch=50)

In [10]:
model.save_model('/content/drive/MyDrive/Semester/MTP/Phishing-Detection/phishing.custom.embedding.unsupervised.200.bin')

# Word2vec

In [ ]:
sentences = [row.split(".") for row in processed_url_df['processed_url']]
sentences_tokenized = [sublist[0].split() for sublist in sentences]

In [ ]:
model_word2vec = Word2Vec(sentences_tokenized, min_count=1, vector_size=100, window=3, sg=0)

In [ ]:
model_word2vec.save('/content/drive/MyDrive/Semester/MTP/Phishing-Detection/phishing.word2vec.embedding.100.bin')

# Glove

In [ ]:
# !wget http://nlp.stanford.edu/data/glove.6B.zip

In [ ]:
# !unzip -q glove.6B.zip